In [1]:
!pip install datasets
!pip install transformers
!pip install accelerate
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is i

In [2]:
!pip install dashscope
!pip install modelscope_studio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 60.7 MB/s eta 0:00:00


In [5]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# ...


Device set to use cpu


KeyboardInterrupt: 

In [ ]:
import time
import torch
from transformers import pipeline
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
system_message = {"role": "system", "content": ("You are a Premium Chatbot who thinks before speaking and uses 2 paragraphs: one for Reasoning and another for Response. When replying, format your answer using the following tags exactly: output your reasoning with '<|assistant reasoning|>' followed by your chain-of-thought explanation, and then output your final reply with '<|assistant response|>' followed by your answer. Also, please use lowercase for '<|system|>' and '<|user|>' tags.")}
conversation = [system_message]
def reformat_output(output_text: str) -> str:
    reformatted = output_text.replace("<System>", "<|system|>").replace("<User>", "<|user|>")
    if "<assistant>" in reformatted:
        before, assistant_block = reformatted.split("<assistant>", 1)
        if "Reasoning:" in assistant_block and "Answer:" in assistant_block:
            reasoning_part = assistant_block.split("Answer:")[0]
            response_part = assistant_block.split("Answer:")[1]
            reasoning_part = reasoning_part.replace("Reasoning:", "").strip()
            response_part = response_part.strip()
            new_assistant_text = f"<|assistant reasoning|> {reasoning_part}\n<|assistant response|> {response_part}"
            reformatted = before + new_assistant_text
        else:
            reformatted = reformatted.replace("<assistant>", "<|assistant reasoning|>")
    return reformatted
print("Type 'bye' to exit the chat.")
while True:
    user_input = input("User: ")
    if user_input.strip().lower() == "bye":
        print("Chatbot: Farewell, Sir!")
        break
    conversation.append({"role": "user", "content": user_input})
    prompt = pipe.tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    start_time = time.time()
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    end_time = time.time()
    response_time = end_time - start_time
    generated_text = outputs[0]["generated_text"]
    formatted_output = reformat_output(generated_text)
    print("Chatbot:")
    print(formatted_output)
    print(f"\nResponse Time: {response_time:.2f} seconds")
    print("\n" + "-"*50 + "\n")
    conversation.append({"role": "assistant", "content": generated_text})


In [ ]:
import time
import torch
import gradio as gr
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
system_message = {"role": "system", "content": ("You are a Premium Chatbot who thinks before speaking and uses 2 paragraphs: one for Reasoning and another for Response. When replying, format your answer using the following tags exactly: output your reasoning with '<|assistant reasoning|>' followed by your chain-of-thought explanation, and then output your final reply with '<|assistant response|>' followed by your answer. Also, please use lowercase for '<|system|>' and '<|user|>' tags.")}
conversation = [system_message]

def reformat_output(output_text: str) -> str:
    reformatted = output_text.replace("<System>", "<|system|>").replace("<User>", "<|user|>")
    if "<assistant>" in reformatted:
        before, assistant_block = reformatted.split("<assistant>", 1)
        if "Reasoning:" in assistant_block and "Answer:" in assistant_block:
            reasoning_part = assistant_block.split("Answer:")[0]
            response_part = assistant_block.split("Answer:")[1]
            reasoning_part = reasoning_part.replace("Reasoning:", "").strip()
            response_part = response_part.strip()
            new_assistant_text = f"<|assistant reasoning|> {reasoning_part}\n<|assistant response|> {response_part}"
            reformatted = before + new_assistant_text
        else:
            reformatted = reformatted.replace("<assistant>", "<|assistant reasoning|>")
    return reformatted

def chat(user_input, state):
    if user_input.strip().lower() == "bye":
        return "Chatbot: Farewell, Sir!", state
    state.append({"role": "user", "content": user_input})
    prompt = pipe.tokenizer.apply_chat_template(state, tokenize=False, add_generation_prompt=True)
    start_time = time.time()
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    end_time = time.time()
    response_time = end_time - start_time
    generated_text = outputs[0]["generated_text"]
    formatted_output = reformat_output(generated_text)
    state.append({"role": "assistant", "content": generated_text})
    result = f"{formatted_output}\n\nResponse Time: {response_time:.2f} seconds"
    return result, state

with gr.Blocks() as demo:
    gr.Markdown("## LogicLink Chatbot")
    chatbot_state = gr.State(value=conversation)
    chatbot_output = gr.Textbox(label="Chatbot Response", interactive=False)
    user_input_text = gr.Textbox(placeholder="Type your message here...", label="Your Message")
    send_btn = gr.Button("Send")
    send_btn.click(chat, inputs=[user_input_text, chatbot_state], outputs=[chatbot_output, chatbot_state])

demo.launch()


In [ ]:
def generate_formatted_output(user_prompt):
    system_text = (
        "You are a Premium Chatbot who thinks before speaking and always responds "
        "in the specified format."
    )

    # Reasoning is based on the provided reasoning prompt
    assistant_reasoning = "Different ways to approach the Query : " + user_prompt + " [Insert chain-of-thought reasoning here.]"

    # Response is based on the provided response prompt (echoing the query here)
    assistant_response = user_prompt  # Replace with your actual response logic if needed

    formatted_output = (
        f"<|system|>\n{system_text}\n"
        f"<|user|>\n{user_prompt}\n"
        f"<|assistant reasoning|>\n{assistant_reasoning}\n"
        f"<|assistant response|>\n{assistant_response}"
    )

    return formatted_output

# Example usage:
user_input = "hello how are you"
output = generate_formatted_output(user_input)
print(output)


In [ ]:
import time
import torch
import gradio as gr
from transformers import pipeline

# Initialize the text-generation pipeline
pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

def logiclink_chat(user_input, history):
    if not user_input:
        return history, ""
    # Append user message
    history.append(("You", user_input))

    # Build prompts for reasoning and response
    reasoning_prompt = (
        "Reflect on all potential nuances, underlying concepts, and alternative perspectives "
        "before forming a final answer for the following query: " + user_input
    )
    response_prompt = user_input

    start = time.time()
    # Generate chain-of-thought reasoning
    reasoning_out = pipe(
        reasoning_prompt,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    reasoning = reasoning_out[0]["generated_text"].strip()

    # Generate final answer
    response_out = pipe(
        response_prompt,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    response = response_out[0]["generated_text"].strip()
    elapsed = time.time() - start

    # Append reasoning and response to history
    history.append(("LogicLink – Reasoning", reasoning))
    history.append(("LogicLink - Response", f"{response}\n\n*({elapsed:.2f}s)*"))

    return history, ""

# Build Gradio interface
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=3):
            gr.Markdown("## LogicLink Chat")
            chatbot = gr.Chatbot(label="Conversation", height=600)
            user_input = gr.Textbox(show_label=False, placeholder="Type your query here…")
            send = gr.Button("Send", variant="primary")
        with gr.Column(scale=1):
            gr.Markdown(
                "**How LogicLink Works**\n\n"
                "- You type a question.\n"
                "- LogicLink first shows its _chain-of-thought_.\n"
                "- Then it gives the concise answer below.\n"
                "- Conversation history is on the left."
            )

    send.click(logiclink_chat, [user_input, chatbot], [chatbot, user_input])
    user_input.submit(logiclink_chat, [user_input, chatbot], [chatbot, user_input])

demo.launch()


In [ ]:
pip install -U transformers peft

In [ ]:
import time
import torch
import gradio as gr
from transformers import pipeline

# Initialize the text-generation pipeline
pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

def logiclink_chat(user_input, history):
    if not user_input:
        return history, ""
    # Append user message
    history.append(("You", user_input))

    # Build reasoning prompt
    reasoning_prompt = (
        "Reflect on all potential nuances, underlying concepts, and alternative perspectives "
        "before forming a final answer for the following query: " + user_input
    )

    start = time.time()
    # Generate chain-of-thought reasoning
    reasoning = pipe(
        reasoning_prompt,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )[0]["generated_text"].strip()

    # Generate final answer
    response = pipe(
        user_input,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )[0]["generated_text"].strip()

    elapsed = time.time() - start

    # Append reasoning and response
    history.append(("LogicLink – Reasoning", reasoning))
    history.append(("LogicLink – Response", f"{response}\n\n*({elapsed:.2f}s)*"))

    return history, ""

# Custom CSS for black background, red accents, blue borders
custom_css = """
body {
    background-color: #000;
    color: #fff;
}
.gradio-container {
    border-radius: 8px;
    padding: 1rem;
}
.gr-button, .gr-button:hover {
    background-color: #e53935 !important;  /* red buttons */
    color: #fff !important;
}
.gr-textbox, .gr-chatbot {
    border: 2px solid #1e88e5 !important;  /* blue borders */
}
.gr-textbox textarea, .gr-chatbot {
    background-color: #111;
    color: #fff;
}
.gr-row {
    margin-bottom: 1rem;
}
"""

with gr.Blocks(css=custom_css) as demo:
    with gr.Row():
        with gr.Column(scale=3):
            gr.Markdown("## 🎨 LogicLink Chatbot")
            chatbot = gr.Chatbot(label="Conversation", height=600)
            user_input = gr.Textbox(show_label=False, placeholder="Type your query here…")
            send_btn = gr.Button("Send")
        with gr.Column(scale=1):
            gr.Markdown(
                "**How LogicLink Works**\n\n"
                "- You type a question.\n"
                "- LogicLink shows its _chain-of-thought_ first.\n"
                "- Then it gives the concise answer below.\n"
                "- Chat history stays on the left."
            )

    send_btn.click(logiclink_chat, [user_input, chatbot], [chatbot, user_input])
    user_input.submit(logiclink_chat, [user_input, chatbot], [chatbot, user_input])

demo.launch()


In [ ]:
import uuid
import time
import json
import gradio as gr
import modelscope_studio.components.antd as antd
import modelscope_studio.components.antdx as antdx
import modelscope_studio.components.base as ms
import modelscope_studio.components.pro as pro
import dashscope
from config import DEFAULT_LOCALE, DEFAULT_SETTINGS, DEFAULT_THEME, DEFAULT_SUGGESTIONS, save_history, get_text, user_config, bot_config, welcome_config, api_key, MODEL_OPTIONS_MAP
from ui_components.logo import Logo
from ui_components.settings_header import SettingsHeader
from ui_components.thinking_button import ThinkingButton
from dashscope import Generation

dashscope.api_key = api_key


def format_history(history, sys_prompt):
    # messages = [{
    #     "role": "system",
    #     "content": sys_prompt,
    # }]
    messages = []
    for item in history:
        if item["role"] == "user":
            messages.append({"role": "user", "content": item["content"]})
        elif item["role"] == "assistant":
            contents = [{
                "type": "text",
                "text": content["content"]
            } for content in item["content"] if content["type"] == "text"]
            messages.append({
                "role":
                "assistant",
                "content":
                contents[0]["text"] if len(contents) > 0 else ""
            })
    return messages


class Gradio_Events:

    @staticmethod
    def submit(state_value):

        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]
        settings = state_value["conversation_contexts"][
            state_value["conversation_id"]]["settings"]
        enable_thinking = state_value["conversation_contexts"][
            state_value["conversation_id"]]["enable_thinking"]
        model = settings.get("model")
        messages = format_history(history,
                                  sys_prompt=settings.get("sys_prompt", ""))

        history.append({
            "role":
            "assistant",
            "content": [],
            "key":
            str(uuid.uuid4()),
            "header":
            MODEL_OPTIONS_MAP.get(model, {}).get("label", None),
            "loading":
            True,
            "status":
            "pending"
        })

        yield {
            chatbot: gr.update(value=history),
            state: gr.update(value=state_value),
        }
        try:
            response = Generation.call(
                model=model,
                messages=messages,
                stream=True,
                result_format='message',
                incremental_output=True,
                enable_thinking=enable_thinking,
                thinking_budget=settings.get("thinking_budget", 1) * 1024)
            start_time = time.time()
            reasoning_content = ""
            answer_content = ""
            is_thinking = False
            is_answering = False
            contents = [None, None]
            for chunk in response:
                if (not chunk.output.choices[0].message.get("content")
                        and not chunk.output.choices[0].message.get(
                            "reasoning_content")):
                    pass
                else:
                    delta = chunk.output.choices[0].message
                    if hasattr(
                            delta,
                            'reasoning_content') and delta.reasoning_content:
                        if not is_thinking:
                            contents[0] = {
                                "type": "tool",
                                "content": "",
                                "options": {
                                    "title": get_text("Thinking...", "思考中..."),
                                    "status": "pending"
                                },
                                "copyable": False,
                                "editable": False
                            }
                            is_thinking = True
                        reasoning_content += delta.reasoning_content
                    if hasattr(delta, 'content') and delta.content:
                        if not is_answering:
                            thought_cost_time = "{:.2f}".format(time.time() -
                                                                start_time)
                            if contents[0]:
                                contents[0]["options"]["title"] = get_text(
                                    f"End of Thought ({thought_cost_time}s)",
                                    f"已深度思考 (用时{thought_cost_time}s)")
                                contents[0]["options"]["status"] = "done"
                            contents[1] = {
                                "type": "text",
                                "content": "",
                            }

                            is_answering = True
                        answer_content += delta.content

                    if contents[0]:
                        contents[0]["content"] = reasoning_content
                    if contents[1]:
                        contents[1]["content"] = answer_content
                history[-1]["content"] = [
                    content for content in contents if content
                ]

                history[-1]["loading"] = False
                yield {
                    chatbot: gr.update(value=history),
                    state: gr.update(value=state_value)
                }
            print("model: ", model, "-", "reasoning_content: ",
                  reasoning_content, "\n", "content: ", answer_content)
            history[-1]["status"] = "done"
            cost_time = "{:.2f}".format(time.time() - start_time)
            history[-1]["footer"] = get_text(f"{cost_time}s",
                                             f"用时{cost_time}s")
            yield {
                chatbot: gr.update(value=history),
                state: gr.update(value=state_value),
            }
        except Exception as e:
            print("model: ", model, "-", "Error: ", e)
            history[-1]["loading"] = False
            history[-1]["status"] = "done"
            history[-1]["content"] += [{
                "type":
                "text",
                "content":
                f'<span style="color: var(--color-red-500)">{str(e)}</span>'
            }]
            yield {
                chatbot: gr.update(value=history),
                state: gr.update(value=state_value)
            }
            raise e

    @staticmethod
    def add_message(input_value, settings_form_value, thinking_btn_state_value,
                    state_value):
        if not state_value["conversation_id"]:
            random_id = str(uuid.uuid4())
            history = []
            state_value["conversation_id"] = random_id
            state_value["conversation_contexts"][
                state_value["conversation_id"]] = {
                    "history": history
                }
            state_value["conversations"].append({
                "label": input_value,
                "key": random_id
            })

        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]

        state_value["conversation_contexts"][
            state_value["conversation_id"]] = {
                "history": history,
                "settings": settings_form_value,
                "enable_thinking": thinking_btn_state_value["enable_thinking"]
            }
        history.append({
            "role": "user",
            "content": input_value,
            "key": str(uuid.uuid4())
        })
        yield Gradio_Events.preprocess_submit(clear_input=True)(state_value)

        try:
            for chunk in Gradio_Events.submit(state_value):
                yield chunk
        except Exception as e:
            raise e
        finally:
            yield Gradio_Events.postprocess_submit(state_value)

    @staticmethod
    def preprocess_submit(clear_input=True):

        def preprocess_submit_handler(state_value):
            history = state_value["conversation_contexts"][
                state_value["conversation_id"]]["history"]
            return {
                **({
                    input:
                    gr.update(value=None, loading=True) if clear_input else gr.update(loading=True),
                } if clear_input else {}),
                conversations:
                gr.update(active_key=state_value["conversation_id"],
                          items=list(
                              map(
                                  lambda item: {
                                      **item,
                                      "disabled":
                                      True if item["key"] != state_value[
                                          "conversation_id"] else False,
                                  }, state_value["conversations"]))),
                add_conversation_btn:
                gr.update(disabled=True),
                clear_btn:
                gr.update(disabled=True),
                conversation_delete_menu_item:
                gr.update(disabled=True),
                chatbot:
                gr.update(value=history,
                          bot_config=bot_config(
                              disabled_actions=['edit', 'retry', 'delete']),
                          user_config=user_config(
                              disabled_actions=['edit', 'delete'])),
                state:
                gr.update(value=state_value),
            }

        return preprocess_submit_handler

    @staticmethod
    def postprocess_submit(state_value):
        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]
        return {
            input:
            gr.update(loading=False),
            conversation_delete_menu_item:
            gr.update(disabled=False),
            clear_btn:
            gr.update(disabled=False),
            conversations:
            gr.update(items=state_value["conversations"]),
            add_conversation_btn:
            gr.update(disabled=False),
            chatbot:
            gr.update(value=history,
                      bot_config=bot_config(),
                      user_config=user_config()),
            state:
            gr.update(value=state_value),
        }

    @staticmethod
    def cancel(state_value):
        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]
        history[-1]["loading"] = False
        history[-1]["status"] = "done"
        history[-1]["footer"] = get_text("Chat completion paused", "对话已暂停")
        return Gradio_Events.postprocess_submit(state_value)

    @staticmethod
    def delete_message(state_value, e: gr.EventData):
        index = e._data["payload"][0]["index"]
        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]
        history = history[:index] + history[index + 1:]

        state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"] = history

        return gr.update(value=state_value)

    @staticmethod
    def edit_message(state_value, chatbot_value, e: gr.EventData):
        index = e._data["payload"][0]["index"]
        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]
        history[index]["content"] = chatbot_value[index]["content"]
        return gr.update(value=state_value)

    @staticmethod
    def regenerate_message(settings_form_value, thinking_btn_state_value,
                           state_value, e: gr.EventData):
        index = e._data["payload"][0]["index"]
        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]
        history = history[:index]

        state_value["conversation_contexts"][
            state_value["conversation_id"]] = {
                "history": history,
                "settings": settings_form_value,
                "enable_thinking": thinking_btn_state_value["enable_thinking"]
            }

        yield Gradio_Events.preprocess_submit()(state_value)
        try:
            for chunk in Gradio_Events.submit(state_value):
                yield chunk
        except Exception as e:
            raise e
        finally:
            yield Gradio_Events.postprocess_submit(state_value)

    @staticmethod
    def select_suggestion(input_value, e: gr.EventData):
        input_value = input_value[:-1] + e._data["payload"][0]
        return gr.update(value=input_value)

    @staticmethod
    def apply_prompt(e: gr.EventData):
        return gr.update(value=e._data["payload"][0]["value"]["description"])

    @staticmethod
    def new_chat(thinking_btn_state, state_value):
        if not state_value["conversation_id"]:
            return gr.skip()
        state_value["conversation_id"] = ""
        thinking_btn_state["enable_thinking"] = True
        return gr.update(active_key=state_value["conversation_id"]), gr.update(
            value=None), gr.update(value=DEFAULT_SETTINGS), gr.update(
                value=thinking_btn_state), gr.update(value=state_value)

    @staticmethod
    def select_conversation(thinking_btn_state_value, state_value,
                            e: gr.EventData):
        active_key = e._data["payload"][0]
        if state_value["conversation_id"] == active_key or (
                active_key not in state_value["conversation_contexts"]):
            return gr.skip()
        state_value["conversation_id"] = active_key
        thinking_btn_state_value["enable_thinking"] = state_value[
            "conversation_contexts"][active_key]["enable_thinking"]
        return gr.update(active_key=active_key), gr.update(
            value=state_value["conversation_contexts"][active_key]["history"]
        ), gr.update(value=state_value["conversation_contexts"][active_key]
                     ["settings"]), gr.update(
                         value=thinking_btn_state_value), gr.update(
                             value=state_value)

    @staticmethod
    def click_conversation_menu(state_value, e: gr.EventData):
        conversation_id = e._data["payload"][0]["key"]
        operation = e._data["payload"][1]["key"]
        if operation == "delete":
            del state_value["conversation_contexts"][conversation_id]

            state_value["conversations"] = [
                item for item in state_value["conversations"]
                if item["key"] != conversation_id
            ]

            if state_value["conversation_id"] == conversation_id:
                state_value["conversation_id"] = ""
                return gr.update(
                    items=state_value["conversations"],
                    active_key=state_value["conversation_id"]), gr.update(
                        value=None), gr.update(value=state_value)
            else:
                return gr.update(
                    items=state_value["conversations"]), gr.skip(), gr.update(
                        value=state_value)
        return gr.skip()

    @staticmethod
    def toggle_settings_header(settings_header_state_value):
        settings_header_state_value[
            "open"] = not settings_header_state_value["open"]
        return gr.update(value=settings_header_state_value)

    @staticmethod
    def clear_conversation_history(state_value):
        if not state_value["conversation_id"]:
            return gr.skip()
        state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"] = []
        return gr.update(value=None), gr.update(value=state_value)

    @staticmethod
    def update_browser_state(state_value):

        return gr.update(value=dict(
            conversations=state_value["conversations"],
            conversation_contexts=state_value["conversation_contexts"]))

    @staticmethod
    def apply_browser_state(browser_state_value, state_value):
        state_value["conversations"] = browser_state_value["conversations"]
        state_value["conversation_contexts"] = browser_state_value[
            "conversation_contexts"]
        return gr.update(
            items=browser_state_value["conversations"]), gr.update(
                value=state_value)


css = """
.gradio-container {
  padding: 0 !important;
}
.gradio-container > main.fillable {
  padding: 0 !important;
}
#chatbot {
  height: calc(100vh - 21px - 16px);
  max-height: 1500px;
}
#chatbot .chatbot-conversations {
  height: 100vh;
  background-color: var(--ms-gr-ant-color-bg-layout);
  padding-left: 4px;
  padding-right: 4px;
}
#chatbot .chatbot-conversations .chatbot-conversations-list {
  padding-left: 0;
  padding-right: 0;
}
#chatbot .chatbot-chat {
  padding: 32px;
  padding-bottom: 0;
  height: 100%;
}
@media (max-width: 768px) {
  #chatbot .chatbot-chat {
      padding: 0;
  }
}
#chatbot .chatbot-chat .chatbot-chat-messages {
  flex: 1;
}
#chatbot .setting-form-thinking-budget .ms-gr-ant-form-item-control-input-content {
    display: flex;
    flex-wrap: wrap;
}
"""

model_options_map_json = json.dumps(MODEL_OPTIONS_MAP)
js = "function init() { window.MODEL_OPTIONS_MAP=" + model_options_map_json + "}"

with gr.Blocks(css=css, js=js, fill_width=True) as demo:
    state = gr.State({
        "conversation_contexts": {},
        "conversations": [],
        "conversation_id": "",
    })

    with ms.Application(), antdx.XProvider(
            theme=DEFAULT_THEME, locale=DEFAULT_LOCALE), ms.AutoLoading():
        with antd.Row(gutter=[20, 20], wrap=False, elem_id="chatbot"):
            # Left Column
            with antd.Col(md=dict(flex="0 0 260px", span=24, order=0),
                          span=0,
                          order=1):
                with ms.Div(elem_classes="chatbot-conversations"):
                    with antd.Flex(vertical=True,
                                   gap="small",
                                   elem_style=dict(height="100%")):
                        # Logo
                        Logo()

                        # New Conversation Button
                        with antd.Button(value=None,
                                         color="primary",
                                         variant="filled",
                                         block=True) as add_conversation_btn:
                            ms.Text(get_text("New Conversation", "新建对话"))
                            with ms.Slot("icon"):
                                antd.Icon("PlusOutlined")

                        # Conversations List
                        with antdx.Conversations(
                                elem_classes="chatbot-conversations-list",
                        ) as conversations:
                            with ms.Slot('menu.items'):
                                with antd.Menu.Item(
                                        label="Delete", key="delete",
                                        danger=True
                                ) as conversation_delete_menu_item:
                                    with ms.Slot("icon"):
                                        antd.Icon("DeleteOutlined")
            # Right Column
            with antd.Col(flex=1, elem_style=dict(height="100%")):
                with antd.Flex(vertical=True,
                               gap="small",
                               elem_classes="chatbot-chat"):
                    # Chatbot
                    chatbot = pro.Chatbot(elem_classes="chatbot-chat-messages",
                                          height=0,
                                          welcome_config=welcome_config(),
                                          user_config=user_config(),
                                          bot_config=bot_config())

                    # Input
                    with antdx.Suggestion(
                            items=DEFAULT_SUGGESTIONS,
                            # onKeyDown Handler in Javascript
                            should_trigger="""(e, { onTrigger, onKeyDown }) => {
                      switch(e.key) {
                        case '/':
                          onTrigger()
                          break
                        case 'ArrowRight':
                        case 'ArrowLeft':
                        case 'ArrowUp':
                        case 'ArrowDown':
                          break;
                        default:
                          onTrigger(false)
                      }
                      onKeyDown(e)
                    }""") as suggestion:
                        with ms.Slot("children"):
                            with antdx.Sender(placeholder=get_text(
                                    "Enter \"/\" to get suggestions",
                                    "输入 \"/\" 获取提示"), ) as input:
                                with ms.Slot("header"):
                                    settings_header_state, settings_form = SettingsHeader(
                                    )
                                with ms.Slot("prefix"):
                                    with antd.Flex(
                                            gap=4,
                                            wrap=True,
                                            elem_style=dict(maxWidth='40vw')):
                                        with antd.Button(
                                                value=None,
                                                type="text") as setting_btn:
                                            with ms.Slot("icon"):
                                                antd.Icon("SettingOutlined")
                                        with antd.Button(
                                                value=None,
                                                type="text") as clear_btn:
                                            with ms.Slot("icon"):
                                                antd.Icon("ClearOutlined")
                                        thinking_btn_state = ThinkingButton()

    # Events Handler
    # Browser State Handler
    if save_history:
        browser_state = gr.BrowserState(
            {
                "conversation_contexts": {},
                "conversations": [],
            },
            storage_key="qwen3_chat_demo_storage")
        state.change(fn=Gradio_Events.update_browser_state,
                     inputs=[state],
                     outputs=[browser_state])

        demo.load(fn=Gradio_Events.apply_browser_state,
                  inputs=[browser_state, state],
                  outputs=[conversations, state])

    # Conversations Handler
    add_conversation_btn.click(fn=Gradio_Events.new_chat,
                               inputs=[thinking_btn_state, state],
                               outputs=[
                                   conversations, chatbot, settings_form,
                                   thinking_btn_state, state
                               ])
    conversations.active_change(fn=Gradio_Events.select_conversation,
                                inputs=[thinking_btn_state, state],
                                outputs=[
                                    conversations, chatbot, settings_form,
                                    thinking_btn_state, state
                                ])
    conversations.menu_click(fn=Gradio_Events.click_conversation_menu,
                             inputs=[state],
                             outputs=[conversations, chatbot, state])
    # Chatbot Handler
    chatbot.welcome_prompt_select(fn=Gradio_Events.apply_prompt,
                                  outputs=[input])

    chatbot.delete(fn=Gradio_Events.delete_message,
                   inputs=[state],
                   outputs=[state])
    chatbot.edit(fn=Gradio_Events.edit_message,
                 inputs=[state, chatbot],
                 outputs=[state])

    regenerating_event = chatbot.retry(
        fn=Gradio_Events.regenerate_message,
        inputs=[settings_form, thinking_btn_state, state],
        outputs=[
            input, clear_btn, conversation_delete_menu_item,
            add_conversation_btn, conversations, chatbot, state
        ])

    # Input Handler
    submit_event = input.submit(
        fn=Gradio_Events.add_message,
        inputs=[input, settings_form, thinking_btn_state, state],
        outputs=[
            input, clear_btn, conversation_delete_menu_item,
            add_conversation_btn, conversations, chatbot, state
        ])
    input.cancel(fn=Gradio_Events.cancel,
                 inputs=[state],
                 outputs=[
                     input, conversation_delete_menu_item, clear_btn,
                     conversations, add_conversation_btn, chatbot, state
                 ],
                 cancels=[submit_event, regenerating_event],
                 queue=False)
    # Input Actions Handler
    setting_btn.click(fn=Gradio_Events.toggle_settings_header,
                      inputs=[settings_header_state],
                      outputs=[settings_header_state])
    clear_btn.click(fn=Gradio_Events.clear_conversation_history,
                    inputs=[state],
                    outputs=[chatbot, state])
    suggestion.select(fn=Gradio_Events.select_suggestion,
                      inputs=[input],
                      outputs=[input])

if __name__ == "__main__":
    demo.queue(default_concurrency_limit=100,
               max_size=100).launch(ssr_mode=False, max_threads=100)

In [ ]:
import uuid
import time
import json
import gradio as gr
import torch
from transformers import pipeline
import modelscope_studio.components.antd as antd
import modelscope_studio.components.antdx as antdx
import modelscope_studio.components.base as ms
import modelscope_studio.components.pro as pro
from config import DEFAULT_LOCALE, DEFAULT_THEME, get_text, user_config, bot_config, welcome_config
from ui_components.logo import Logo
from ui_components.settings_header import SettingsHeader

# Initialize TinyLlama pipeline
pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

class Gradio_Events:
    @staticmethod
    def generate_response(prompt):
        """Generate both reasoning and final response using TinyLlama"""
        try:
            # Generate reasoning
            reasoning = pipe(
                f"Reflect on all potential nuances and concepts for: {prompt}",
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
            )[0]["generated_text"].strip()

            # Generate final answer
            response = pipe(
                prompt,
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
            )[0]["generated_text"].strip()

            return reasoning, response
        except Exception as e:
            return None, str(e)

    @staticmethod
    def submit(state_value):
        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]

        try:
            user_input = history[-1]["content"]
            start_time = time.time()
            reasoning, answer = Gradio_Events.generate_response(user_input)

            if not answer:
                raise Exception("Failed to generate response")

            elapsed = time.time() - start_time

            assistant_message = {
                "role": "assistant",
                "content": [
                    {
                        "type": "tool",
                        "content": reasoning,
                        "options": {
                            "title": get_text("Chain of Thought", "思考链"),
                            "status": "done"
                        },
                        "copyable": False,
                        "editable": False
                    },
                    {
                        "type": "text",
                        "content": f"{answer}\n\n*({elapsed:.2f}s)*",
                    }
                ],
                "key": str(uuid.uuid4()),
                "loading": False,
                "status": "done"
            }

            history.append(assistant_message)

        except Exception as e:
            error_message = {
                "role": "assistant",
                "content": [{
                    "type": "text",
                    "content": f'<span style="color: #e53935">Error: {str(e)}</span>'
                }],
                "key": str(uuid.uuid4()),
                "loading": False,
                "status": "done"
            }
            history.append(error_message)

        return {
            "chatbot": gr.update(value=history),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def add_message(input_value, state_value):
        if not input_value.strip():
            return gr.skip()

        if not state_value["conversation_id"]:
            random_id = str(uuid.uuid4())
            state_value["conversation_id"] = random_id
            state_value["conversation_contexts"][random_id] = {"history": []}
            state_value["conversations"].append({
                "label": input_value[:20] + ("..." if len(input_value) > 20 else ""),
                "key": random_id
            })

        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]

        history.append({
            "role": "user",
            "content": input_value,
            "key": str(uuid.uuid4())
        })

        return {
            "input": gr.update(value=""),
            "chatbot": gr.update(value=history),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def new_chat(state_value):
        state_value["conversation_id"] = ""
        return {
            "conversations": gr.update(items=state_value["conversations"]),
            "chatbot": gr.update(value=[]),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def clear_history(state_value):
        if state_value["conversation_id"]:
            state_value["conversation_contexts"][
                state_value["conversation_id"]]["history"] = []
        return {
            "chatbot": gr.update(value=[]),
            "state": gr.update(value=state_value)
        }

# Custom CSS with red/blue/black theme
css = """
:root {
    --color-red: #ff4444;
    --color-blue: #1e88e5;
    --color-black: #000000;
    --color-dark-gray: #121212;
}

.gradio-container {
    background: var(--color-black) !important;
    color: white !important;
}

/* Input styling */
.gr-textbox textarea {
    background: var(--color-dark-gray) !important;
    border: 2px solid var(--color-blue) !important;
    color: white !important;
}

/* Output (chatbot) styling */
.gr-chatbot {
    background: var(--color-dark-gray) !important;
    border: 2px solid var(--color-red) !important;
}

/* User message bubbles */
.gr-chatbot .user {
    background: var(--color-blue) !important;
    border-color: var(--color-blue) !important;
}

/* Assistant message bubbles */
.gr-chatbot .bot {
    background: var(--color-dark-gray) !important;
    border: 1px solid var(--color-red) !important;
}

/* Buttons */
.gr-button {
    background: var(--color-blue) !important;
    border-color: var(--color-blue) !important;
}

/* Thinking tooltip */
.gr-chatbot .tool {
    background: var(--color-dark-gray) !important;
    border: 1px solid var(--color-red) !important;
}
"""
class Gradio_Events:
    # Add state management for generation control
    _generating = False

    @staticmethod
    def generate_response(prompt):
        """Generate both reasoning and final response using TinyLlama"""
        try:
            # Generate reasoning
            reasoning = pipe(
                f"Reflect on all potential nuances and concepts for: {prompt}",
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
            )[0]["generated_text"].strip()

            # Generate final answer
            response = pipe(
                prompt,
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
            )[0]["generated_text"].strip()

            return reasoning, response
        except Exception as e:
            return None, str(e)

    @staticmethod
    def add_message(input_value, state_value):
        if not input_value.strip():
            return gr.skip()

        if not state_value["conversation_id"]:
            random_id = str(uuid.uuid4())
            state_value["conversation_id"] = random_id
            state_value["conversation_contexts"][random_id] = {"history": []}
            state_value["conversations"].append({
                "label": input_value[:20] + ("..." if len(input_value) > 20 else ""),
                "key": random_id
            })

        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]

        history.append({
            "role": "user",
            "content": input_value,
            "key": str(uuid.uuid4())
        })

        return {
            "input": gr.update(value=""),
            "chatbot": gr.update(value=history),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def submit(state_value):
        if Gradio_Events._generating:
            return gr.skip()

        Gradio_Events._generating = True
        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]

        try:
            user_input = history[-1]["content"]
            start_time = time.time()
            reasoning, answer = Gradio_Events.generate_response(user_input)

            if not answer:
                raise Exception("Failed to generate response")

            elapsed = time.time() - start_time

            assistant_message = {
                "role": "assistant",
                "content": [
                    {
                        "type": "tool",
                        "content": reasoning,
                        "options": {
                            "title": get_text("Chain of Thought", "思考链"),
                            "status": "done"
                        },
                        "copyable": False,
                        "editable": False
                    },
                    {
                        "type": "text",
                        "content": f"{answer}\n\n*({elapsed:.2f}s)*",
                        "style": {"color": "#ff4444"}
                    }
                ],
                "key": str(uuid.uuid4()),
                "loading": False,
                "status": "done"
            }

            history.append(assistant_message)

        except Exception as e:
            error_message = {
                "role": "assistant",
                "content": [{
                    "type": "text",
                    "content": f'<span style="color: #ff4444">Error: {str(e)}</span>'
                }],
                "key": str(uuid.uuid4()),
                "loading": False,
                "status": "done"
            }
            history.append(error_message)

        Gradio_Events._generating = False
        return {
            "chatbot": gr.update(value=history),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def new_chat(state_value):
        state_value["conversation_id"] = ""
        return {
            "conversations": gr.update(items=state_value["conversations"]),
            "chatbot": gr.update(value=[]),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def clear_history(state_value):
        if state_value["conversation_id"]:
            state_value["conversation_contexts"][
                state_value["conversation_id"]]["history"] = []
        return {
            "chatbot": gr.update(value=[]),
            "state": gr.update(value=state_value)
        }

with gr.Blocks(css=css, fill_width=True) as demo:
    state = gr.State({
        "conversation_contexts": {},
        "conversations": [],
        "conversation_id": "",
    })

    with ms.Application(), antdx.XProvider(
            theme=DEFAULT_THEME, locale=DEFAULT_LOCALE), ms.AutoLoading():
        with antd.Row(gutter=[20, 20], wrap=False, elem_id="chatbot"):
            # Left Column
            with antd.Col(md=dict(flex="0 0 260px", span=24, order=0),
                        span=0, order=1):
                with ms.Div(elem_classes="chatbot-conversations"):
                    with antd.Flex(vertical=True, gap="small",
                                  elem_style=dict(height="100%")):
                        Logo()

                        # New Chat Button
                        with antd.Button(
                            color="primary",
                            variant="filled",
                            block=True,
                            elem_classes="new-chat-btn"
                        ) as new_chat_btn:
                            ms.Text(get_text("New Chat", "新建对话"))
                            with ms.Slot("icon"):
                                antd.Icon("PlusOutlined")

                        # Conversations List
                        with antdx.Conversations(
                                elem_classes="chatbot-conversations-list"
                        ) as conversations:
                            with ms.Slot('menu.items'):
                                with antd.Menu.Item(
                                        label="Delete",
                                        key="delete",
                                        danger=True
                                ) as conversation_delete_menu_item:
                                    with ms.Slot("icon"):
                                        antd.Icon("DeleteOutlined")

            # Right Column
            with antd.Col(flex=1, elem_style=dict(height="100%")):
                with antd.Flex(vertical=True, gap="small",
                               elem_classes="chatbot-chat"):
                    # Chat Display
                    chatbot = pro.Chatbot(
                        elem_classes="chatbot-chat-messages",
                        height=600,
                        welcome_config=welcome_config(),
                        user_config=user_config(),
                        bot_config=bot_config()
                    )

                    # Input Area
                    with antdx.Suggestion(items=[]):
                        with ms.Slot("children"):
                            with antdx.Sender(
                                placeholder="Type your message...",
                                elem_classes="chat-input"
                            ) as input:
                                with ms.Slot("prefix"):
                                    with antd.Flex(gap=4):
                                        with antd.Button(
                                            type="text",
                                            elem_classes="clear-btn"
                                        ) as clear_btn:
                                            with ms.Slot("icon"):
                                                antd.Icon("ClearOutlined")

    # Event Handlers
    input.submit(
        fn=Gradio_Events.add_message,
        inputs=[input, state],
        outputs=[input, chatbot, state]
    ).then(
        fn=Gradio_Events.submit,
        inputs=[state],
        outputs=[chatbot, state]
    )

    new_chat_btn.click(
        fn=Gradio_Events.new_chat,
        inputs=[state],
        outputs=[conversations, chatbot, state]
    )

    clear_btn.click(
        fn=Gradio_Events.clear_history,
        inputs=[state],
        outputs=[chatbot, state]
    )

demo.queue().launch()

In [1]:
import uuid
import time
import gradio as gr
import torch
from transformers import pipeline
import modelscope_studio.components.antd as antd
import modelscope_studio.components.antdx as antdx
import modelscope_studio.components.base as ms
import modelscope_studio.components.pro as pro
from config import (
    DEFAULT_LOCALE, DEFAULT_THEME, get_text,
    user_config, bot_config, welcome_config,
    DEFAULT_SUGGESTIONS
)
from ui_components.logo import Logo
from ui_components.settings_header import SettingsHeader

# Initialize TinyLlama pipeline
pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

class Gradio_Events:
    _generating = False

    @staticmethod
    def generate_response(prompt):
        try:
            reasoning = pipe(
                f"Reflect on all potential nuances and concepts for: {prompt}",
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
            )[0]["generated_text"].strip()
            response = pipe(
                prompt,
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
            )[0]["generated_text"].strip()
            return reasoning, response
        except Exception as e:
            return None, str(e)

    @staticmethod
    def add_message(input_value, state_value):
        if not input_value.strip():
            return gr.skip()
        # create new conversation if needed
        if not state_value["conversation_id"]:
            cid = str(uuid.uuid4())
            state_value["conversation_id"] = cid
            state_value["conversation_contexts"][cid] = {"history": []}
            state_value["conversations"].append({
                "label": "New Chat",
                "key": cid
            })
        hist = state_value["conversation_contexts"][state_value["conversation_id"]]["history"]
        hist.append({
            "role": "user",
            "content": input_value,
            "key": str(uuid.uuid4())
        })
        return {
            "input": gr.update(value=""),
            "chatbot": gr.update(value=hist),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def submit(state_value):
        if Gradio_Events._generating:
            return gr.skip()
        Gradio_Events._generating = True

        hist = state_value["conversation_contexts"][state_value["conversation_id"]]["history"]
        user_input = hist[-1]["content"]
        start = time.time()
        reasoning, answer = Gradio_Events.generate_response(user_input)
        if answer is None:
            hist.append({
                "role": "assistant",
                "content": [{
                    "type": "text",
                    "content": f'<span style="color:red">Error generating response</span>'
                }],
                "key": str(uuid.uuid4()),
                "loading": False,
                "status": "done"
            })
        else:
            elapsed = time.time() - start
            hist.append({
                "role": "assistant",
                "content": [
                    {
                        "type": "tool",
                        "content": reasoning,
                        "options": {
                            "title": get_text("Chain of Thought", "思考链"),
                            "status": "done"
                        },
                        "copyable": False,
                        "editable": False
                    },
                    {
                        "type": "text",
                        "content": f"{answer}\n\n*({elapsed:.2f}s)*"
                    }
                ],
                "key": str(uuid.uuid4()),
                "loading": False,
                "status": "done"
            })

        Gradio_Events._generating = False
        return {
            "chatbot": gr.update(value=hist),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def new_chat(state_value):
        state_value["conversation_id"] = ""
        return {
            "conversations": gr.update(items=state_value["conversations"]),
            "chatbot": gr.update(value=[]),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def clear_history(state_value):
        cid = state_value["conversation_id"]
        if cid:
            state_value["conversation_contexts"][cid]["history"] = []
        return {
            "chatbot": gr.update(value=[]),
            "state": gr.update(value=state_value)
        }


# Custom CSS
css = """
:root {
  --color-red: #ff4444;
  --color-blue: #1e88e5;
  --color-black: #000000;
  --color-dark: #121212;
}
.gradio-container {
  background: var(--color-black) !important;
  color: #fff !important;
}
.gr-textbox textarea {
  background: var(--color-dark) !important;
  border: 2px solid var(--color-blue) !important;
  color: #fff !important;
}
.gr-chatbot {
  background: var(--color-dark) !important;
  border: 2px solid var(--color-red) !important;
}
.gr-chatbot .user {
  background: var(--color-blue) !important;
  border-color: var(--color-blue) !important;
}
.gr-chatbot .bot {
  background: var(--color-dark) !important;
  border: 1px solid var(--color-red) !important;
}
.gr-button {
  background: var(--color-blue) !important;
  border-color: var(--color-blue) !important;
}
"""


with gr.Blocks(css=css, fill_width=True) as demo:
    # Initialize an empty conversation
    initial_id = str(uuid.uuid4())
    state = gr.State({
        "conversation_contexts": { initial_id: {"history": []} },
        "conversations": [ {"label": "New Chat", "key": initial_id} ],
        "conversation_id": initial_id
    })

    with ms.Application(), antdx.XProvider(theme=DEFAULT_THEME, locale=DEFAULT_LOCALE), ms.AutoLoading():
        with antd.Row(gutter=[20, 20], wrap=False, elem_id="chatbot"):
            # Sidebar
            with antd.Col(md=dict(flex="0 0 260px", span=24), span=0):
                with ms.Div(elem_classes="chatbot-conversations"):
                    with antd.Flex(vertical=True, gap="small", elem_style={"height": "100%"}):
                        Logo()
                        with antd.Button(color="primary", variant="filled", block=True) as new_btn:
                            ms.Text(get_text("New Chat", "新对话"))
                            with ms.Slot("icon"):
                                antd.Icon("PlusOutlined")
                        with antdx.Conversations(elem_classes="chatbot-conversations-list") as convs:
                            with ms.Slot("menu.items"):
                                with antd.Menu.Item(label="Delete", key="delete", danger=True) as del_item:
                                    with ms.Slot("icon"):
                                        antd.Icon("DeleteOutlined")

            # Chat area
            with antd.Col(flex=1, elem_style={"height": "100%"}):
                with antd.Flex(vertical=True, gap="small", elem_classes="chatbot-chat"):
                    # Pro Chatbot display
                    chatbot = pro.Chatbot(
                        elem_classes="chatbot-chat-messages",
                        height=600,
                        welcome_config=welcome_config(),
                        user_config=user_config(),
                        bot_config=bot_config()
                    )
                    # Suggestion + Input
                    with antdx.Suggestion(items=DEFAULT_SUGGESTIONS) as suggestion:
                        with ms.Slot("children"):
                            with antdx.Sender(
                                placeholder=get_text("Type message...", "输入消息..."),
                                elem_classes="chat-input"
                            ) as input_box:
                                with ms.Slot("prefix"):
                                    with antd.Flex(gap=4):
                                        with antd.Button(type="text", elem_classes="clear-btn") as clear_btn:
                                            with ms.Slot("icon"):
                                                antd.Icon("ClearOutlined")

                    # Make default suggestions clickable
                    suggestion.select(
                        lambda val: gr.update(value=val),
                        inputs=[],
                        outputs=[input_box]
                    )
                    # Make welcome prompts clickable
                    chatbot.welcome_prompt_select(
                        lambda choice: gr.update(value=choice["description"]),
                        outputs=[input_box]
                    )

    # Event handlers
    input_box.submit(
        Gradio_Events.add_message, [input_box, state],
        [input_box, chatbot, state]
    ).then(
        Gradio_Events.submit, [state],
        [chatbot, state]
    )
    new_btn.click(
        Gradio_Events.new_chat, [state],
        [convs, chatbot, state]
    )
    clear_btn.click(
        Gradio_Events.clear_history, [state],
        [chatbot, state]
    )

if __name__ == "__main__":
    demo.queue().launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1018: UserWarning: Expected 1 arguments for function <function <lambda> at 0x7f4f2e548400>, received 0.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1022: UserWarning: Expected at least 1 arguments for function <function <lambda> at 0x7f4f2e548400>, received 0.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1018: UserWarning: Expected 1 arguments f

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eea847c1133292358b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
